# Plot amplitude decay with distance from dipole in four sphere head model and infinite homogeneous medium

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from lfpykit.eegmegcalc import FourSphereVolumeConductor
from brainsignals.plotting_convention import mark_subplots, simplify_axes, cmap_v_e

head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]

radii = [89000., 90000., 95000., 100000.]  # (µm)
sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)

cdm_amp = 100
p = np.array([[0.], [0.], [cdm_amp]]) # 1 timestep (nA µm)
dipole_loc = np.array([0., 0., 88000.])  # (µm)

z = np.linspace(dipole_loc[2] + 500, radii[-1] - 1, 500)
x = np.zeros(len(z))
y = np.zeros(len(z))
r_elecs = np.array([x, y, z]).T # (µm)

def plot_four_sphere_model(ax, radii, dipole_loc=None, labeltext=True):

    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), radius=radii[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='k', lw=.1))
    if not dipole_loc is None:
        ax.arrow(dipole_loc[0], dipole_loc[2] - 1500, 0, 750, 
                 color='k', head_width=500)
    
    if labeltext:
        # mark 4-sphere head model layers
        ax.text(-17500, radii[0] - 4500, 'brain', ha="right", rotation=8)
        ax.text(-17500, radii[1] - 3300, 'CSF', ha="right", rotation=8)
        ax.text(-17500, radii[2] - 4500, 'skull', ha="right", rotation=8)
        ax.text(-17500, radii[3] - 5000, 'scalp', ha="right", rotation=8)

        
def dipole_potential(elec_locs, dipole_pos, p):
    # Potential in infinite homogeneous medium
    r_ = elec_locs.T - dipole_pos
    V_e = 1000 * 1. / (4 * np.pi * sigmas[0]) * (np.dot(r_, p.T)
                    / np.linalg.norm(r_, axis=1) ** 3)
    return V_e
        

In [ ]:
sphere_model = FourSphereVolumeConductor(r_elecs, radii, sigmas)
M = sphere_model.get_transformation_matrix(dipole_loc)  
pot_4s_decay = 1000 * M @ p # (uV)

# Compare with potential from dipole in infinite homogeneous medium
pot_homog_decay = dipole_potential(r_elecs.T, dipole_loc, p[:, 0])

In [ ]:
xlim = [-17000, 17000]
zlim = [dipole_loc[2] + 100, radii[-1] - 1]

# Many point will give long computation times
num_angles = 61
num_heights_eeg = 50

max_angle = np.abs(np.rad2deg(np.arcsin(xlim[0] / zlim[0])))
angle = np.linspace(-max_angle, max_angle, num_angles)

x_eeg = np.zeros((num_heights_eeg, num_angles))
z_eeg = np.zeros((num_heights_eeg, num_angles))

for r_idx, r in enumerate(np.linspace(zlim[0], zlim[1], num_heights_eeg)):
    x_ = r * np.sin(np.deg2rad(angle))
    z_ = r * np.cos(np.deg2rad(angle))
    x_eeg[r_idx, :] = x_
    z_eeg[r_idx, :] = z_

y_eeg = np.zeros(x_eeg.shape)
r_elecs_grid = np.array([x_eeg.flatten(), y_eeg.flatten(), z_eeg.flatten()]).T

pot_homog = dipole_potential(r_elecs_grid.T, dipole_loc, p[:, 0])

four_sphere = FourSphereVolumeConductor(r_elecs_grid, radii, sigmas)
M = four_sphere.get_transformation_matrix(dipole_loc)  
pot_four_sphere = 1000 * M @ p # (uV)


### Next, we simulate how the EEG changes as the population is moved laterally along the cortical surface

In [ ]:
num_pops = 100
phi = np.linspace(0, np.pi / 3, num_pops)
dipole_radial_dist = dipole_loc[2]
pop_ys = np.zeros(num_pops)
pop_xs = dipole_radial_dist * np.sin(phi)
pop_zs = dipole_radial_dist * np.cos(phi)

num_elecs = len(z_)
Fs_4s = np.zeros(num_pops)
Fs_ih = np.zeros(num_pops)
sphere_model_ = FourSphereVolumeConductor(r_elecs[-1:,:], radii, sigmas)
for p_idx in range(num_pops):

    dipole_loc_ = np.array([pop_xs[p_idx], 
                           pop_ys[p_idx], 
                           pop_zs[p_idx]])
    radial_vec = dipole_loc_ / np.linalg.norm(dipole_loc_)
    
    M = sphere_model_.get_transformation_matrix(dipole_loc_)
    Fs_4s[p_idx] = 1000 * M @ radial_vec * cdm_amp
    Fs_ih[p_idx] = dipole_potential(r_elecs[-1:,:].T, dipole_loc_, radial_vec * cdm_amp)

In [ ]:
ylim = [dipole_loc[2] - 3000, radii[-1] + 500]

fig = plt.figure(figsize=[6, 4.5])
fig.subplots_adjust(bottom=0.08, right=0.98, left=0.03, top=0.95, 
                    wspace=.18, hspace=0.6)
ax_geom = fig.add_subplot(321, frameon=False, aspect=1,
                         xticks=[], yticks=[],
                         xlim=[-17000, 17000],
                         ylim=ylim)
ax_pot = fig.add_subplot(322, xlabel="radial distance (mm)", ylabel="potential (µV)")

plot_four_sphere_model(ax_geom, radii, dipole_loc)
ax_geom.plot(x, z, '--', lw=1, c='k')

l1, = ax_pot.loglog((z - dipole_loc[2]) / 1000, pot_4s_decay[:, 0], c='k')
l3, = ax_pot.loglog((z - dipole_loc[2]) / 1000, pot_homog_decay, c='0.8')

layer_dist_from_neuron = [(r - dipole_loc[2])*1e-3 for r in radii]
for ax in [ax_pot]:
    ax.axvspan(0, layer_dist_from_neuron[0], facecolor=head_colors[0])
    ax.axvspan(layer_dist_from_neuron[0], layer_dist_from_neuron[1], facecolor=head_colors[1])
    ax.axvspan(layer_dist_from_neuron[1], layer_dist_from_neuron[2], facecolor=head_colors[2])
    ax.axvspan(layer_dist_from_neuron[2], layer_dist_from_neuron[3], facecolor=head_colors[3])

fig.legend([l3, l1], ["infinite homogeneous", "four-sphere", ], 
           frameon=False, ncol=2, loc=(0.53, 0.95))
simplify_axes(ax_pot)
mark_subplots(fig.axes, ypos=1.04, xpos=-0.075)


num = 35
levels = np.logspace(-6, 0, num=num)
scale_max = 1#10**np.ceil(np.log10(np.max(np.abs(pot_homog))))
levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2)) 
                   for i in range(len(levels_norm) - 1)]

colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

pot_homog = pot_homog.reshape(x_eeg.shape)
pot_foursphere = pot_four_sphere.reshape(x_eeg.shape)

ax_homog = plt.subplot(323, aspect=1, xlim=xlim, ylim=ylim, 
                       title="infinite homogeneous",
                       frameon=False, xticks=[], yticks=[])

ax_foursphere = plt.subplot(324, aspect=1, xlim=xlim, ylim=ylim, 
                            title="four-sphere", 
                            frameon=False, xticks=[], yticks=[])

axes = [ax_homog, ax_foursphere]
pots = [pot_homog, pot_foursphere]

for i in range(len(axes)):
    ax = axes[i]
    pot = pots[i]
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), radius=radii[i], ls="--",
                                   fill=False, ec='gray', lw=1.))

    ax.arrow(dipole_loc[0], dipole_loc[2] - 1500, 0, 750,
                 color='k', head_width=500)
    
    img = ax.contourf(x_eeg, z_eeg, pot,
                               zorder=-2, colors=colors_from_map,
                               levels=levels_norm, extend='both')

    ax.contour(x_eeg, z_eeg, pot, linewidths = 0.4,
               colors='k', zorder = -2,
               levels=levels_norm)

cax = fig.add_axes([0.47, 0.42, 0.01, 0.2], frameon=False)
cbar = fig.colorbar(img, cax=cax,
            #orientation='horizontal', 
                    label="µV")#format='%3.3f',)

ticks = [-1e0, -1e-2, -1e-4, 0, 1e-4, 1e-2, 1e0]
ticklabels = ["-1", "-10⁻²","-10⁻⁴", "0", "10⁻⁴", "10⁻²", "1"]
cbar.set_ticks(ticks)
cbar.set_ticklabels(ticklabels)

mark_subplots(axes, "CD", xpos=.0, ypos=1.0)

ylim_ = [dipole_loc[2] - 50000, radii[-1] + 500]
xlim_ = [-99000, 99000]
ax_geom = fig.add_subplot(325, frameon=False, aspect=1,
                         xticks=[], yticks=[],
                         xlim=xlim_,
                         ylim=ylim_)

ax_pot = fig.add_subplot(326,
                         xlabel=r"$l_{\rm arc}$ (cm)", 
                         ylabel="potential (µV)")

plot_four_sphere_model(ax_geom, radii, labeltext=False)

ax_geom.plot(x[-1], z[-1], '_', c='k')
ax_geom.plot(pop_xs, pop_zs, 'r--')


l1, = ax_pot.semilogy(dipole_radial_dist * phi * 1e-4, Fs_4s, c='k')
l2, = ax_pot.semilogy(dipole_radial_dist * phi * 1e-4, Fs_ih, c='0.8')

x_lambda = 2.2
x_ = dipole_radial_dist * phi * 1e-4
y_ = Fs_4s[0] * np.exp(-x_**1 / x_lambda)
l0, = ax_pot.plot(x_, y_, c='b')

mark_subplots([ax_geom], ["E"], xpos=0.1, ypos=1.1)
mark_subplots([ax_pot], ["F"])

ax_pot.grid(True)

fig.legend([l2, l1, l0], ["infinite homogeneous", 
                          "four-sphere", 
                          r"$F_0 e^{-l_{\rm arc}/(2.2{\rm cm})}$"], 
           frameon=False, ncol=2, loc=(0.52, 0.3))

simplify_axes(ax_pot)
plt.savefig("pot_decay_head.pdf")